In [13]:
import pathlib
import os, sys
import numpy as np
from loguru import logger
from collections import defaultdict
from openset_imagenet.util import ccr_at_fpr
import openset_imagenet
from matplotlib import pyplot

In [14]:
def load_scores(args):
    # we sort them as follows: protocol, loss, algorithm
    scores = defaultdict(lambda: defaultdict(dict))
    ground_truths = {}
    for net in args["networks"]:
                output_directory = pathlib.Path(args["output_directory"])
                score_file = f"{net}.npz"
                if os.path.exists(score_file):
                    # remember files
                    results = np.load(score_file)
                    scores[net] = results["scores"] # only change in maxlogits case

                    if len(ground_truths) == 0:
                        ground_truths = results["gt"].astype(int)
                    else:
                        assert np.all(results["gt"] == ground_truths)

                    logger.info(f"Loaded score file {score_file} for net {net}")
                else:
                    logger.warning(f"Did not find score file {score_file} for net {net}")

    return scores, ground_truths

In [15]:
arguments = { 
  "output_directory": "experiments/ex_7/net_13",
  "networks": ["net_13"]
  }
THRESHOLDS = {
              1e-3: "$10^{-3}$",
              1e-2: "$10^{-2}$",
              1e-1: "$10^{-1}$",
              1: "$1$",
}

scores, ground_truths = load_scores(arguments)
# we get ccr@fpr for each network
for net in arguments["networks"]:
    ccrs = ccr_at_fpr(ground_truths, scores[net], THRESHOLDS, unk_label=-2)
    print(f"Network {net}: {ccrs} for thresholds {THRESHOLDS.values()}")

2024-06-18 09:08:31.673 | INFO     | __main__:load_scores:18 - Loaded score file net_13.npz for net net_13


Network net_13: [None, 0.14933333333333335, 0.38466666666666666, 0.6193333333333333] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
